## Write out Cube Transit file from routed GTFS

In [ ]:
import os
import pandas as pd

from network_wrangler import TransitNetwork
from lasso.seflorida.seflorida_transit import SEFloridaTransit
from lasso.seflorida.seflorida_parameters import SEFloridaParameters

In [ ]:
print("setup basic paths...")
root_dir = "projects/serpm/"
input_dir = os.path.join(root_dir, "data", "interim", "8-standard-format")
gtfs_input_dir = os.path.join(root_dir, "data", "interim", "6-transit-standard-reroute")
output_dir = os.path.join(root_dir, "data", "processed")

print("read parameters...")
lasso_dir = os.path.join(root_dir, "Lasso")
parameters = SEFloridaParameters(lasso_base_dir=lasso_dir)

In [ ]:
# create standard transit network from routed gtfs files
transit_net = TransitNetwork.read(feed_path = gtfs_input_dir)
standard_transit_net = SEFloridaTransit(feed=transit_net.feed, parameters = parameters)

In [ ]:
# create unique, consecutive shape_id
unique_shape_ids = standard_transit_net.feed.trips.copy()
unique_shape_ids = (
    unique_shape_ids.groupby(["agency_raw_name", "shape_id"])["trip_id"]
    .count()
    .reset_index()
    .drop(["trip_id"], axis=1)
)
unique_shape_ids["shape_id_original"] = unique_shape_ids["shape_id"]
unique_shape_ids["shape_id"] = range(1, len(unique_shape_ids) + 1)
unique_shape_ids["shape_id"] = unique_shape_ids["shape_id"].astype(str)

temp_shapes = standard_transit_net.feed.shapes.copy()
temp_shapes = temp_shapes.rename(columns={"shape_id": "shape_id_original"})
temp_shapes = pd.merge(temp_shapes, unique_shape_ids, how="left", on=["agency_raw_name", "shape_id_original"])
standard_transit_net.feed.shapes = temp_shapes.copy()

standard_transit_net.feed.trips = standard_transit_net.feed.trips.rename(columns={"shape_id": "shape_id_original"})
standard_transit_net.feed.trips = pd.merge(standard_transit_net.feed.trips, temp_shapes[["shape_id_original", "shape_id"]].drop_duplicates(), how="left", on="shape_id_original")

In [ ]:
# create numeric, consecutive route_id
temp_routes = standard_transit_net.feed.routes.copy().drop(columns="agency_id")
temp_routes = temp_routes.rename(columns={"route_id": "route_id_original"})
temp_routes["route_id"] = range(1, len(temp_routes) + 1)
standard_transit_net.feed.routes = temp_routes.copy()

standard_transit_net.feed.trips = standard_transit_net.feed.trips.rename(columns={"route_id": "route_id_original"})
standard_transit_net.feed.trips = pd.merge(standard_transit_net.feed.trips, temp_routes[["route_id_original", "route_id"]].drop_duplicates(), how="left", on="route_id_original")

In [ ]:
standard_transit_net.feed.trips = standard_transit_net.feed.trips.drop(columns=["trip_num_x", "trip_num_y"])
standard_transit_net.feed.trips["trip_num"] = standard_transit_net.feed.trips["trip_num"].astype(float).astype(int)
standard_transit_net.feed.frequencies = standard_transit_net.feed.frequencies[["trip_id", "headway_secs", "start_time", "end_time"]]
standard_transit_net.feed.trips["agency_id"] = standard_transit_net.feed.trips["agency_raw_name"]
standard_transit_net.feed.agency["agency_id"] = standard_transit_net.feed.agency["agency_raw_name"]

In [ ]:
# get operator code from previous routing result
route_operator_code_lookup = pd.read_csv(f"{root_dir}/data/interim/6-transit-standard/routes.txt")
route_operator_code_lookup = route_operator_code_lookup[["route_id_original", "operator_code"]]
standard_transit_net.feed.routes = pd.merge(standard_transit_net.feed.routes, route_operator_code_lookup, how="left", on="route_id_original")
standard_transit_net.feed.routes.head()

In [ ]:
# fill MDT_local operator
standard_transit_net.feed.routes.loc[standard_transit_net.feed.routes["operator_code"].isnull(), "operator_code"] = "MDT_local"
# check if all routes have operator_code
na_operator = standard_transit_net.feed.routes[standard_transit_net.feed.routes["operator_code"].isnull()]
na_operator

In [ ]:
# write result as cube format
standard_transit_net.write_as_cube_lin(parameters=parameters, outpath = f"{output_dir}/transit/transit_rerouted.lin")